# Ising Model in 1D and 2D: Metropolis Monte Carlo Simulations

In this lab, we'll be studying the [Ising model](https://en.wikipedia.org/wiki/Ising_model), a simple model of magnetization in solid crystals, that been mapped to many other physical, biological and sociological phenomena. The goal is for you to come away with some familiarity with how magnetic solids are formed and how to use simulations to the study a physical process.

In the ising model, we consider atoms at lattice points in a crystal with spin $s$, which can take on values of $s=\pm1$. They are often represented as arrows like this
![spins](images/spin_up_spin_down.png)

As we have discussed in our lectures on atomic physics spins give particles agular momentum, which in turn give them a magentic moment. In the Ising Model each spin (i) interacts with only its neighbors (j), with an interaction energy
$$ U_{ij} = - s_i s_j$$

If we put a few spins in a line (a 1-D crystal), our system starts to looks like this.
![fig1](images/1d_fig1.png)

The squares below the spins are an alternate representation of our model (black => $s=+1$ grey => $s=-1$).


We can visualize the interactions of a particular atom by looking how it is aligned relative to its neighbors. The selected atom has two favorable interactions depicted in green. It would have a energy of $U=-2$.
![fig2](images/1d_fig2.png)

When studying a solid, physics typically like to understand what happens in the bulk material. However, on a computer, we are limited to study a finite collection of atoms (or we would need an infinite amount of memory!), so we can cheat a little bit by using periodic boundary conditions to remove edges from our system.

![fig3](images/1d_fig3.png)

In this example the atom would have $U = 0$, because it is aligned with one of its neighbors and anti-aligned with the other.


In the lab, we'll also do some simulations in 2D. In that case, our system will look like a grid of atoms, each interacting with its 8 neighbors.

![fig4](images/2d_fig1.png)

The depicted atom is this case would have $U = -2$ because it is aligned with 5 of its neighbors and anti-aligned with 3 of them. Like in 1-D we use periodic boundary conditions, so the top connects to the bottom and the left connects to the right.

The problem we study with the Ising model is understanding why at some temperatures a ferromagnet (like the ones on your fridge), would demagnetize (or "melt"), while at other temperatures, nearly every spin in the entire magnet is aligned (producing a bulk magnetic dipole moment). To do this, we connect the model we've define here to a thermal bath - a source of random thermal energy.

## The Ising model at finite temperature: a crash course in statistical physics

You can think of a "thermal bath" as an infinitely large block of material and a temperature $T$. Any thing that is coupled to it, will eventually also come to equilibrium at temperature $T$.

If we imagine our magnet is placed in contact with a thermal bath, our atoms will now be subjected to a barrage of random energetic fluctions; the higher the temperature the more energy these flucuations have.

We adopt a probabilistic view of our model where the probability of a given state being observed P is proportional to the negative exponentional of its energy.

$$ P(s_1, s_2, ...) \propto \exp(-U(s_1, s_2, ...) / k_BT)$$

This is called the Boltzmann distribution and $k_B$ is the Boltzmann constant. For convenience this is often rewritten with $\beta = 1/k_BT$ so

$$ P \propto \exp(-\beta U) $$

As our system evolves, it will sample different "microstates" - specific configurations of spins. At the human scale the precise configuration of spins does not matter, as much as the the average characteristics of the spin configuration behavior. These "macrostates" represent a collection of microstates with similar properties.

As a system grows to infinite size, the relatively probabilities of different macrostates grow dramatically, and the system typically evolves into a particular macrostate.

In our Ising-model system, we'll describe the macrostates in terms of the average energy of atoms in the crystal, and the magnetization (the sum of all the spins). If most atoms are aligned with their neighbors, the the energy will be low. If most atoms are aligned over the entire crystal, the magnetization will have a finite value.


$$ U = \sum_{ij} U_{ij} = - \sum s_i s_j$$

$$ M = \sum_i s_i$$


## Metropolis Monte Carlo

It's not possible to solve for the behavior of the Ising model in closed form. However, we can use computers to simulate its behavior. To do that we'll use the [Metropolis Monte Carlo](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) algorithm to sample different microstates and then observe the macrostates by computing U and M.

We'll gloss of the the details, but it proceeds basically by a weighted trial and error procedure that is guaranteed to evolve your model into thermal equilibrium for infinite trials (but you only need to do a finite number).

First, you specify a random configuration of spins. This is your initial state, then

1. Randomly select a spin and flip it ($s_{new} = - s_{old}$)
1. If this move lowers the total system energy U, accept it. If it doesn't accept it with $P = \exp-\beta\Delta U$
1. Repeat. Each of cycle is called a "move." A "sweep" is when you have attempted to flip every spin in your system

After many sweeps, your system is in thermal equilbrium. As you continue to make moves, you'll sample different microstates that correspond the the equilibrium macrostate.

This notebook contains code to run MC simulations of the Ising model in one and two dimensions. As we run the model, we'll compute its energy and magnetization every so often, and take snapshots of the model's microstate. Rather than using the arrow representation, we'll used black and white images to represent the spins.


## Local ordering and Spatial Autocorrelations


Fially, I wanted to introduce you to another way of charaterizing the microstructure of a solid (or fluid). At many temperatures you will find that the magnetization will be zero (or very close to it). However, the microstates you see will look different to your eyes - you'll see "grains."

These are regions of local order - clumps of aligned spins. If we'd like to characterize their size, we can use something called the spatial autocorrelation function.

We'll start in 1D by defining our spin field S(x). It's just the spin at the lattice site x.

$$ S(x_i) = s_i$$

The autocorrelation is defined by

$$ R(\Delta x) = \sum_x S(x+\Delta x) S(x) $$

If, on average, a spin at a distance $\Delta x$ away from any lattice point, has the same spin, this function will be positive. If, on average, they are not correlated, it will be zero. If, on average, the spin is in the the opposite direction, it will be negative. If 2Dthe definitely is basically the same, but we perform a radial average, over some separation $\Delta r$ between lattice sites.

If we look at a result from a 2 dimensional simulation with M=0 and look at its radial distribution function we can see that as the separation distance $\Delta r$ grows, the autocorrelation function decays. The width of that decay is set by the "grain size" in the simulation.

![sim](images/sim1.png)

![ac](images/radial_ac.png)

**Execute the cells below and follow the instructions.**

In [ ]:
#the first time you run this, you may need to install these packages
# !pip install pillow h5py cython --user

In [ ]:
# import the required libaries
import ipywidgets as widgets
import numpy as np
from ipywidgets import interact
from PIL import Image
from numpy.fft import rfft2, irfft2, fftshift
from numpy.fft import rfft, irfft
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import IPython.display as display
import h5py

%load_ext cython
%matplotlib notebook

In [ ]:
#define some helper functions
def autocorrelation_1D(img):
    fft_img = rfft(img)
    xs = np.arange(len(img)) - 0.5 * len(img)
    return xs, fftshift(irfft(fft_img * np.conj(fft_img)))/np.prod(img.shape)

def display_ising_sequence_1D(images):
    def _show(frame=(0, len(images) - 1)):
        return display_spin_field(images[frame])
    return interact(_show)

def random_spin_field_1D(N):
    return np.random.choice([-1, 1], size=(N))

def magnetization(img):
    return np.sum(img)/np.prod(img.shape)

def display_spin_field_1D(field):
    img = np.array([field]* 40)
    img = Image.fromarray(np.uint8((1 - img) * 0.4 * 255 + 25.5))  # 0 ... 255
    img.save('temp_1d.png')
    with open('temp_1d.png', 'rb') as input_file:
        img = input_file.read()
    return widgets.Image(value=img, layout=widgets.Layout(width='90%'))

def autocorrelation(img):
    fft_final = rfft2(img)
    return fftshift(irfft2(fft_final * np.conj(fft_final)))
    
def radial_autocorrelation(img):
    ac = autocorrelation(img)
    shape = img.shape
    max_r = int(np.min(shape) / np.sqrt(2)) + 1
    hist = np.zeros(max_r + 1)
    counts = np.zeros(max_r + 1)
    for i in range(shape[0]):
        for j in range(shape[1]):
            r = int(np.sqrt((i - 0.5 * shape[0]) ** 2 + (j - 0.5 * shape[1])**2))
            hist[r] += ac[i, j]
            counts[r] += 1
            
    rhist = hist[counts > 0]/counts[counts > 0] / (np.prod(img.shape))
    rs = np.arange(0, max_r)
    return rs, rhist

def display_ising_sequence(images, sweeps, eng, mag, ac_ax, mag_ax, e_ax):
    def _show(sweep):
        ac_ax.clear()
        rs, ac = radial_autocorrelation(images[sweep])
        
        ac_ax.plot(rs, ac)
        ac_ax.set_title('Radial Autocorrelation')
        ac_ax.set_xlabel('$\Delta r$')
        ac_ax.set_ylabel('Autocorrelation')
        ac_ax.set_ylim(-0.5, 1.2)
        
        e_ax.clear()
        e_ax.plot(sweeps, eng, '-k')
        e_ax.plot([sweeps[sweep]], [eng[sweep]], 'or')
        e_ax.set_title('Average Energy  = {:0.2f}'.format(eng[sweep]))
        e_ax.set_xlabel('MC sweeps')
        e_ax.set_ylabel('Energy')
        e_ax.set_ylim(-8, 0)
        
        mag_ax.clear()
        mag_ax.plot(sweeps, mag, '-k')
        mag_ax.plot([sweeps[sweep]], [mag[sweep]], 'or')
        mag_ax.set_title('Magnetization = {:0.2f}'.format(mag[sweep]))
        mag_ax.set_xlabel('MC sweeps')
        mag_ax.set_ylabel('Magentization')
        mag_ax.set_ylim(-1, 1)

        return display_spin_field(images[sweep])
    
    sweep_selector = widgets.SelectionSlider(
        options=[('{:d}'.format(sweep), s) for s, sweep in enumerate(sorted(sweeps))],
        description='sweep = ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        layout=widgets.Layout(width='80%')
    )
    return interact(_show, sweep=sweep_selector)

def display_ising_sequence_1D(images, sweeps, eng, mag, ac_ax, mag_ax, e_ax):
    def _show(sweep):
        ac_ax.clear()
        xs, ac = autocorrelation_1D(images[sweep])
        
        ac_ax.plot(xs, ac)
        ac_ax.set_title('1-D Autocorrelation')
        ac_ax.set_xlabel('$\Delta x$')
        ac_ax.set_ylabel('Autocorrelation')
        ac_ax.set_ylim(-0.5, 1.2)
        
        e_ax.clear()
        e_ax.plot(sweeps, eng, '-k')
        e_ax.plot([sweeps[sweep]], [eng[sweep]], 'or')
        e_ax.set_title('Average Energy  = {:0.2f}'.format(eng[sweep]))
        e_ax.set_xlabel('MC sweeps')
        e_ax.set_ylabel('Energy')
        e_ax.set_ylim(-2, 0)
        
        mag_ax.clear()
        mag_ax.plot(sweeps, mag, '-k')
        mag_ax.plot([sweeps[sweep]], [mag[sweep]], 'or')
        mag_ax.set_title('Magnetization = {:0.2f}'.format(mag[sweep]))
        mag_ax.set_xlabel('MC sweeps')
        mag_ax.set_ylabel('Magentization')
        mag_ax.set_ylim(-1, 1)
        
        return display_spin_field_1D(images[sweep])
    
    sweep_selector = widgets.SelectionSlider(
        options=[('{:d}'.format(sweep), s) for s, sweep in enumerate(sorted(sweeps))],
        description='sweep = ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        layout=widgets.Layout(width='80%')
    )
    return interact(_show, sweep=sweep_selector)

def random_spin_field(N, M):
    return np.random.choice([-1, 1], size=(N, M))

def display_spin_field(field):
    img = Image.fromarray(np.uint8((1 - field) * 0.4 * 255 + 25.25))
    img.save('temp_2d.png')
    with open('temp_2d.png', 'rb') as input_file:
        img = input_file.read()
    return widgets.Image(value=img, layout=widgets.Layout(width='60%'))


In [ ]:
%%cython
# define simulation routines
cimport cython

import numpy as np
cimport numpy as np

from libc.math cimport exp
from libc.stdlib cimport rand
cdef extern from "limits.h":
    int RAND_MAX


@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_step_1D(np.int64_t[:] field, float beta=0.4, int n_moves=1):
    cdef int N = field.shape[0]
    cdef int n_offset, n
    for i in range(n_moves):
        n_offset = np.random.choice(np.arange(2))
        for n in range(n_offset, N, 2):
            _cy_ising_update_1D(field, n, beta)
    return np.array(field)


@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_ising_update_1D(np.int64_t[:] field, int n, float beta):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int i
    for i in range(n-1, n+2):
        if i == n:
            continue
        total += field[i % N]
    cdef float dE = 2 * field[n] * total
    
    if dE <= 0:
        field[n] *= -1
    elif exp(-dE * beta) * RAND_MAX > rand():
        field[n] *= -1



@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_step(np.int64_t[:, :] field, float beta=0.4, int n_moves=1):
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int n_offset, m_offset, n, m
    n_offsets = np.random.choice(range(2), n_moves)
    m_offsets = np.random.choice(range(2), n_moves)
    for (n_offset, m_offset) in zip(n_offsets, m_offsets):
        for n in range(n_offset, N, 2):
            for m in range(m_offset, M, 2):
                _cy_ising_update(field, n, m, beta)
    return np.array(field)


@cython.boundscheck(False)
@cython.wraparound(False)
cdef _cy_ising_update(np.int64_t[:, :] field, int n, int m, float beta):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i, j
    for i in range(n-1, n+2):
        for j in range(m-1, m+2):
            if i == n and j == m:
                continue
            total += field[i % N, j % M]
    cdef float dE = 2 * field[n, m] * total
    if dE <= 0:
        field[n, m] *= -1
    elif exp(-dE * beta) * RAND_MAX > rand():
        field[n, m] *= -1
        
@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_energy(np.int64_t[:, :] field):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int M = field.shape[1]
    cdef int i, j
    cdef int n, m
    cdef float total_E = 0
    cdef float row_E = 0 
    for n in range(N):
        row_E = 0 
        for m in range(M):
            total = 0
            for i in range(n-1, n+2):
                for j in range(m-1, m+2):
                    if i == n and j == m:
                        continue
                    total += field[i % N, j % M]
            row_E += - field[n, m] * total
        total_E += row_E
    return total_E /(N * M)


@cython.boundscheck(False)
@cython.wraparound(False)
def cy_ising_energy_1D(np.int64_t[:] field):
    cdef int total = 0
    cdef int N = field.shape[0]
    cdef int i
    cdef int n
    cdef float total_E = 0
    for n in range(N):
        total = 0
        for i in range(n-1, n+2):
            if i == n:
                continue
            total += field[i % N]
        total_E += - total * field[n]
    return total_E /(N)


In [ ]:
# define simulation wrappers
def ising_1d_simulation(beta, number_of_samples = 100, moves_per_sweep = 1000):
    images = []
    step = random_spin_field_1D(500)
    images.append(step)

    sweep_number = np.arange(0, (number_of_samples + 1) * moves_per_sweep, moves_per_sweep)
    magnet = np.empty(number_of_samples + 1)
    energy = np.empty(number_of_samples + 1)
    magnet[0] = magnetization(step)
    energy[0] = cy_ising_energy_1D(step)

    progress = widgets.FloatProgress(
        value=1,
        min=0,
        max=number_of_samples + 1,
        step=0.1,
        description='Simulating!',
        bar_style='info',
        orientation='horizontal',
        layout=widgets.Layout(width='90%')
    )
    display.display(progress)
    for sweep in range(1, number_of_samples + 1):
        progress.value = sweep
        step = cy_ising_step_1D(step.copy(), beta=beta, n_moves=moves_per_sweep)
        images.append(step)
        energy[sweep] = cy_ising_energy_1D(step)
        magnet[sweep] = magnetization(step)
    
    display.clear_output()
    spec = GridSpec(ncols=12, nrows=12)
    fig = plt.figure(figsize=(9, 9))
    ax1 = fig.add_subplot(spec[0:7,:])
    ax2 = fig.add_subplot(spec[9:, 0:5])
    ax3 = fig.add_subplot(spec[9:,7:])

    
    display_ising_sequence_1D(images, sweep_number, energy, magnet, ax1, ax2, ax3);
    return dict(
        images=images, 
        number_of_samples=number_of_samples,
        moves_per_sweep=moves_per_sweep,
        beta=beta,
    )

def ising_2d_simulation(beta, number_of_samples = 100, moves_per_sweep = 1000, size=150):
    images = []
    step = random_spin_field(size, size)
    images.append(step)

    sweep_number = np.arange(0, (number_of_samples + 1) * moves_per_sweep, moves_per_sweep)
    magnet = np.empty(number_of_samples + 1)
    energy = np.empty(number_of_samples + 1)
    magnet[0] = magnetization(step)
    energy[0] = cy_ising_energy(step)
    
    progress = widgets.FloatProgress(
        value=1,
        min=0,
        max=number_of_samples + 1,
        step=0.1,
        description='Simulating!',
        bar_style='info',
        orientation='horizontal',
        layout=widgets.Layout(width='90%')
    )
    display.display(progress)
    for sweep in range(1, number_of_samples + 1):
        progress.value = sweep
        step = cy_ising_step(step.copy(), beta=beta, n_moves=moves_per_sweep)
        images.append(step)
        energy[sweep] = cy_ising_energy(step)
        magnet[sweep] = magnetization(step)

    display.clear_output()
    spec = GridSpec(ncols=12, nrows=12)
    fig = plt.figure(figsize=(9, 9))
    ax1 = fig.add_subplot(spec[0:7,:])
    ax2 = fig.add_subplot(spec[9:, 0:5])
    ax3 = fig.add_subplot(spec[9:,7:])

    
    display_ising_sequence(images, sweep_number, energy, magnet, ax1, ax2, ax3);
    return dict(
        images=images, 
        number_of_samples=number_of_samples,
        moves_per_sweep=moves_per_sweep,
        beta=beta,
    )

def save_data(data, filename):
    with h5py.File('simulations/' + filename, 'w') as outf:
        outf.attrs.moves_per_sweep = data['moves_per_sweep']
        outf.attrs.number_of_samples = data['number_of_samples']
        outf.images = data['images']
        outf.attrs.beta = data['beta']

## 1 D simulations

Let's get a handle for how the the system behaves in one dimension. Run a few simulations at different temperatures ($\beta = 1/k_BT$). Choose a few values for `beta` in the range of 0.1 to 20.

* Identify a $\beta_m$ where you consistently see your simulation magnetize.
* What is $\beta_m$?
* Is your system mostly spin up or spin down?
* Describe how the energy evolves as you make more MC sweeps.
* What does the correlation function look like that the beginning of this simulation? What does it look like at the end?

When you have answers to the question above, you can enter them in this cell.

In [ ]:
filename = '1D_magnetized.hdf5'
data = ising_1d_simulation(beta=4)
save_data(data, filename=filename)

## 2 D simulations

Now we're going to run a few simulations at different $\beta$s

First, run a simulation for $\beta = 0.15$
* Is your system mostly spin up or spin down?
* Describe how the energy and magentization evolves as you make more MC sweeps. Discuss any behavior you find interesting.
* What does the correlation function look like that the beginning of this simulation? What does it look like at the end?

Write your responses here

In [ ]:
# This will take ~60 seconds to complete.
# A plot will appear when it's done
filename = '2D_exp_1.hdf5'
data = ising_2d_simulation(beta=0.15, moves_per_sweep=1000)
save_data(data, filename=filename)

Next, run a simulation for $\beta = 0.19$
* Is your system mostly spin up or spin down?
* Describe how the energy and magentization evolves as you make more MC sweeps. Discuss any behavior you find interesting.
* What does the correlation function look like that the beginning of this simulation? What does it look like at the end?

Write your responses here

In [ ]:
# This will take ~60 seconds to complete.
# A plot will appear when it's done
filename = '2D_exp_2.hdf5'
data = ising_2d_simulation(beta=0.19, moves_per_sweep=1000)
save_data(data, filename=filename)

Lastly, run a simulation for $\beta = 0.23$
* Is your system mostly spin up or spin down?
* Describe how the energy and magentization evolves as you make more MC sweeps. Discuss any behavior you find interesting.
* What does the correlation function look like that the beginning of this simulation? What does it look like at the end?

Write your responses here

In [ ]:
# This will take ~60 seconds to complete.
# A plot will appear when it's done
filename = '2D_exp_3.hdf5'
data = ising_2d_simulation(beta=0.23, moves_per_sweep=1000)
save_data(data, filename=filename)

Now close this notebook and open the "Lab 7 Data Analysis" notebook (File->Open)